Steps to NLP for ML:

1) Raw text - mdoel cannot read text

2) Tokenizing - telling model what to read from file 

3) Clean text - remove stop words, punctuation, stemming, etc.

4) Vectorize - turn data to numeric form

5) Machine Learning - put into model for train/test
    - classification model

## 1) Preprocess Data

In [ ]:
import pandas as pd
